# Vorbereitung Google Dev Board

Lesen und Folgen Sie Kapitel 3-4 des folgenden Links:\n
https://coral.ai/docs/dev-board/get-started/#install-mdt

Wenn Sie sich mit ihrem Board verbunden haben, können Sie mit dem alias "coral"\
die vordefinierte Python-umgebung aktivieren.\
Mit:

    jupyter-lab --ip=xxx.xxx.xxx.xxx
    
Können Sie jupyter-lab starten. Die Ip-adresse bekommen Sie mittels "ip a" heraus.
Nun sollten Sie einen Link in der Kommandozeile finden, den Sie auf ihrem lokalen Rechner
in den Browser eingeben können. 


# Quantisierung

Bevor wir ein Netzwerk auf dem Dev-Board ausfuehren koennen muessen wir Besonderheiten beachten. \
Die Edge-TPU kann nur quantisierte Netzwerke ausfuehren. \
Wenn Sie ein NN auf ihrem lokalen Rechner trainieren, sind dessen Gewichte typischerweise im float32 format.\
Das Dev-Board verlangt Gewichte im uint8 format. Hier muss also eine Quantisierung von 32 bit auf 8 bit stattfinden. \
Sie Muessen ihr Netzwerk also quantisieren. 
Hierbei gibt es zwei Optionen:

1. Quantization-aware training Trainingsschichten werden mittels ”fake” quantization nodes auf die 8-bit Gewichte der spaeteren Quantisierung vorbereitet.
Daraus resultiert im allgemeinen eine hoehere Genauigkeit im Vergleich zum
post-training.

2. Full integer post-training quantization Jedes beliebige NN kann nach dem
training quantisiert werden. Es wird allerdings eine "helperfunktion" benoetig,
die der Quantisierungsroutine einen Beispielinput übergibt.

Wir verwenden hier Variante 2


In [1]:
# Laden des Models!

Modelname = "Cnn"
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model(Modelname)

model.summary()

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_____________________________________________

 
 Wir benötigen nur ein paar Beispieldaten für die Quantisierung\
 Hierbei wird der Prozess des Vorhersagens ein paar mal durchgespielt \
 und die Gewichte demensprechend beschnitten\
 Am Ende erhalten wir ein Model, das Gewichte im uint8 Format besitzt\
 Vorteil hierbei ist, dass die Vorhersage schneller abgewickelt wird (Auf Kosten der Genauigkeit)\



In [2]:
# Quantisierung

modelname_quant = "model_quant.tflite"

from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
X_train=x_train[0:100] / 255
X_train=np.reshape(X_train, (X_train.shape[0],1,28,28,1))
del x_train, y_train, x_test, y_test




def representative_data_gen_CNN():
    for (label,img) in enumerate(X_train):     
        yield [img.astype(np.float32)]


# Kompiliertes und trainiertes Modell uebergeben
converter = tf.lite.TFLiteConverter.from_keras_model(model)


# Quantisierung aktivieren
converter.optimizations = [tf.lite.Optimize.DEFAULT]


# Hier den Generator einbinden
converter.representative_dataset = representative_data_gen_CNN


# Wirf Fehlermeldung, falls Operation nicht convertiert werden kann
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]


# Quantisierung auf int8 setzen
converter.target_spec.supported_types = [tf.int8]


# Input/Output auf uint8
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8


# Konvertieren
tflite_model = converter.convert()


# Speichern
with open(modelname_quant, 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpiffitogi/assets


Für die Edge-TPU reicht dies allerdings noch nicht.\
Das Netzwerk muss erst noch durch den TPU-Compiler übersetzte werden.
Auf einem Debian-basierten OS kann dieser mittels folgender Befehle installiert werden:

---

    curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

    echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" |
    sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

    sudo apt-get update

    sudo apt-get install edgetpu-compiler


Ausführen:

    edgetpu-compiler modelname.tflite

---

Alternativ kann ein Dockerimage dazu verwendet werden:

---

Im Notebook:

    run_inside_docker = "\"cd data && edgetpu_compiler {modelname}\""\
    .format(modelname = modelname_quant)

    !docker run -v $(pwd):/data -it sichrist/edge_tpu_compiler bash -c \
        {run_inside_docker}
    
---

Das Modell kann innerhalb des Notebooks mit folgendem Befehl auf das Board gepusht werden:

    !mdt push {modelname_to_push} Pfad/zum/Verzeichnis/